##Import SCAPE and other packages and connect to database.  

##NOTE: sometimes you have to type your password in the notebook server terminal, not here in the notebook (a python2/python3 difference).

In [2]:
import scape.registry as R
import scape.sql as sql
import os,json
from getpass import getuser,getpass

import sqlalchemy
from sqlalchemy import Column, Integer, String, Numeric, DATETIME
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

ImportError: No module named 'scape'

In [7]:
#Prepare the URL with authentication data

pwd = getpass()
myurl = sqlalchemy.engine.url.URL(
    'postgresql',         # SQL backend type
    username='postgres',  # or get_user() if username is the same
    password = pwd,       #  
    host='localhost',     # assuming you're on a workstation
    database='lanldb',    # Name of your LANL database
    port=5432)            # Make sure this matches your database configuration

········


In [8]:
# Create a persistent engine
global engine
engine=sqlalchemy.create_engine( myurl )

In [9]:
def connect(engine):
    """Read the registry file and connect the metadata to the database object"""
    with open('lanldataregistry.json','rt') as f:
        overlay = json.load(f)
    
    tmd={k:sql.SqlDataSource(engine=engine,
                             metadata=R.TableMetadata(v),
                             table=k,
                             description='') for k,v in overlay.items()}
    reg = R.Registry(tmd)
    return reg

In [10]:
registry=connect(engine)

## Examples of of using registry to explore and understand data 

In [11]:
#Get one of the data streams
dns=registry['dns']

In [12]:
#inspect the knowledge engineering on it
dns

Field,Dim,Tags
id,primarykey,
resolvedhost,hostname,dnsdnsresponsedst
shost,hostname,src
time,time,seconds


In [1]:
# Select all the data and have a look at it in a pandas dataframe
q1=dns.select().run().dataframe
q1

NameError: name 'dns' is not defined

In [15]:
# Ask SCAPE to return to you only fields that have dimension hosthame
q2=dns.select("hostname").run().dataframe
q2

,resolvedhost,shost
0,C5030,C4653
1,C16712,C5782
2,C419,C1191
3,C22841,C3380
4,C5030,C2436
5,C2109,C161
6,C528,C5642
7,C22841,C3380
8,C1065,C2428
9,C2109,C2428


In [16]:
q1.groupby(["shost"]).size().sort_values()

NameError: name 'q1' is not defined

In [11]:
# Get some quick statistics to find out which host asks a lot of DNS questions in an hour
q1.groupby(["shost"]).size().sort_values(ascending=False)

shost
C3380     133
C5778      80
C2492      74
C5919      69
C5913      46
C23716     25
C768       21
C5875      16
C5917      16
C1191      15
C2428      15
C2303      14
C1936      13
C2320      12
C421       10
C814        9
C608        9
C843        8
C3554       8
C3165       8
C154        7
C2438       7
C3198       7
C1653       7
C831        7
C4291       7
C3026       7
C453        7
C3143       6
C2798       6
         ... 
C3254       2
C4215       1
C3714       1
C354        1
C1905       1
C1142       1
C3607       1
C23724      1
C5782       1
C5642       1
C5633       1
C5358       1
C5337       1
C5332       1
C5186       1
C3131       1
C4233       1
C5043       1
C14343      1
C2668       1
C4812       1
C1800       1
C2032       1
C1754       1
C4517       1
C282        1
C2077       1
C2087       1
C2893       1
C15182      1
dtype: int64

In [17]:
# Ask SCAPE to filter to a particular host, figure out what hosts the talkative host is resolving
q3=dns.select().where("hostname=='C3380'").run().dataframe
q3

,id,time,shost,resolvedhost
0,3,15,C3380,C22841
1,7,38,C3380,C22841
2,30,76,C3380,C22841
3,38,100,C3380,C22841
4,47,137,C3380,C22841
5,60,162,C3380,C22841
6,72,198,C3380,C22841
7,89,224,C3380,C22841
8,102,259,C3380,C22841
9,110,286,C3380,C22841


In [18]:
# Wow, lots of the resolutions go to a particular outside host.  Let's count up
q3.groupby(["resolvedhost"]).size().sort_values(ascending=False)

resolvedhost
C22841    120
C2013       2
C798        1
C586        1
C5728       1
C5640       1
C5588       1
C5136       1
C457        1
C25295      1
C25294      1
C22529      1
C22167      1
dtype: int64

In [19]:
#Who else is resolving that host?
q4=dns.select().where("hostname=='C22841'").run().dataframe
q4

,id,time,shost,resolvedhost
0,3,15,C3380,C22841
1,7,38,C3380,C22841
2,30,76,C3380,C22841
3,38,100,C3380,C22841
4,47,137,C3380,C22841
5,60,162,C3380,C22841
6,72,198,C3380,C22841
7,89,224,C3380,C22841
8,102,259,C3380,C22841
9,110,286,C3380,C22841


In [20]:
q4.groupby(["shost"]).size().sort_values(ascending=False)
#so I guess nobody else asked for it.

shost
C3380    120
dtype: int64

In [21]:
# Pivot to flows and see if that host is talking to any other internal hosts
flows=registry['flows']

In [22]:
flows.select().where("hostname =='C3380'").run().dataframe

,id,time,duration,shost,sport,dhost,dport,protocol,pktcount,bytecount


In [23]:
flows.select().where("hostname =='C22841'").run().dataframe

,id,time,duration,shost,sport,dhost,dport,protocol,pktcount,bytecount


In [17]:
## Hmm, neither one talked at all...  Let's see if there are interesting flows in this
## Actually, no, I'm going to check it in and go to bed... 